In [1]:
import h5py
import scipy
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from scipy.ndimage.filters import gaussian_filter
from scipy import spatial
import json
import torch
from multiprocessing import Pool
from functools import partial
import cv2
import matplotlib.pyplot as plt

In [ ]:
get_ipython().system_raw("unrar x /content/drive/MyDrive/MobileCount/Visdrone2020-CrowdCounting.rar")

In [ ]:
def gaussian_filter_density(gt):
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    tree = spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1] + distances[i][2] + distances[i][3]) * 0.1
        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [ ]:
def process(img_paths):

  idx = 0
  while idx < len(img_paths):
    img_path = img_paths[idx]
    mat = io.loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground_truth').replace('img', 'GT_img'))

    img = plt.imread(img_path)

    gt = mat["image_info"][0, 0][0, 0][0]

    # Resize dell'immagine
    '''if img.shape[1] >= img.shape[0]:  # Se la larghezza è maggiore dell'altezza (rettangolo in orizzontale)

        # Normalizzazione immagine
        rate_1 = 960.0 / img.shape[1]
        rate_2 = 540 / img.shape[0]

        # Con la resize diventa (width,height)
        # (0,0) è la dimensione minima dell'immagine
        # fx,fy il fattore di scaling dell'immagine
        img = cv2.resize(img, (0, 0), fx=rate_1, fy=rate_2)

        cv2.imwrite(img_path, img)

        # Resize valori del GT
        gt[:, 0] = gt[:, 0] * rate_1
        gt[:, 1] = gt[:, 1] * rate_2


    elif img.shape[0] > img.shape[1]:  # Se l'altezza è maggiore della larghezza (rettangolo in verticale)

        # Normalizzazione immagine
        rate_1 = 960.0 / img.shape[0]
        rate_2 = 540.0 / img.shape[1]

        # Con la resize diventa (width,height)
        # (0,0) è la dimensione minima dell'immagine
        # fx,fy il fattore di scaling dell'immagine
        img = cv2.resize(img, (0, 0), fx=rate_2, fy=rate_1)

        cv2.imwrite(img_path, img)

        # Resize valori del GT
        gt[:, 0] = gt[:, 0] * rate_2
        gt[:, 1] = gt[:, 1] * rate_1
'''

    k = np.zeros((img.shape[0], img.shape[1]))

    for i in range(0, len(gt)):
        # Controllo se le coordinate della persona sono nelle dimensioni dell'immagine
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            # Metto 1 nella matrice di zeri che corrisponde alla posizione di una persona
            k[int(gt[i][1]), int(gt[i][0])] = 1

    k = gaussian_filter_density(k)

    with h5py.File(img_path.replace('.jpg', '.h5').replace('images', 'ground'), 'w') as hf:
        hf['density'] = k
    
    idx += 1

In [ ]:
import os.path
from os import path

if __name__ == '__main__':

    img_paths = []

    # Cartelle che contengono h5
    if path.exists('/content/Visdrone_leggero/part_B/train_data/ground') == False:
      os.mkdir('/content/Visdrone_leggero/part_B/train_data/ground')
    
    if path.exists('/content/Visdrone_leggero/part_B/test_data/ground') == False:
      os.mkdir('/content/Visdrone_leggero/part_B/test_data/ground')


    # Path delle immagini
    for dirpath, dirs, files in os.walk(
            '/content/Visdrone_leggero/part_B/train_data/images'):
        for filename in files:
            fname = os.path.join(dirpath, filename)
            if fname.endswith('.jpg'):
                img_paths.append(fname)

    for dirpath, dirs, files in os.walk(
            '/content/Visdrone_leggero/part_B/test_data/images'):
        for filename in files:
            fname = os.path.join(dirpath, filename)
            if fname.endswith('.jpg'):
                img_paths.append(fname)

    process(img_paths)

generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate density...
done.
generate den

In [ ]:
!zip -r /content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/Visdrone2020CC/dataset_1920x1080/Visdrone_1920x1080-with_h5.zip /content/Visdrone_leggero

In [ ]:
from google.colab import files
files.download("/content/visdrone.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>